# ImageNet A/B/E Benchmark (Colab)

This notebook prepares ImageNet-1k (official route), then runs A/B/E param-matched benchmarks using paper-style training flags.

Prerequisites:
- You have access to ILSVRC2012 (ImageNet-1k) and placed these files in Google Drive:
  - `ILSVRC2012_img_train.tar`
  - `ILSVRC2012_img_val.tar`
  - `ILSVRC2012_devkit_t12.tar`



In [ ]:
# Clone and install
!pip -q install -U pip
!if [ ! -d MoP ]; then git clone https://github.com/Eran-BA/MoP.git ; fi
%cd MoP/
!pip -q install -r requirements.txt


In [ ]:
# Mount Drive and configure paths
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
# Prefer your Drive folder with the big Kaggle zip
IMNET_SRC = '/content/drive/MyDrive/imagenet_directdownload_kaggle'
if not os.path.isdir(IMNET_SRC):
    # Fallback to legacy path if your folder doesn't exist
    IMNET_SRC = '/content/drive/MyDrive/imagenet'
IMNET_ROOT = '/content/imagenet'               # extraction target
print('Source:', IMNET_SRC, '\nTarget:', IMNET_ROOT)
os.makedirs(IMNET_ROOT, exist_ok=True)


In [ ]:
# Use existing ZIP from Drive (skip Kaggle)
import os, glob, zipfile, tarfile

# IMNET_SRC is set in the previous cell, prefering /content/drive/MyDrive/imagenet_directdownload_kaggle
os.makedirs(IMNET_SRC, exist_ok=True)

# Find large zip(s) in IMNET_SRC
zips = sorted(glob.glob(os.path.join(IMNET_SRC, '*.zip')), key=os.path.getsize, reverse=True)
print('Found zips:', [os.path.basename(z) for z in zips])

if zips:
    outer_zip = zips[0]
    print('Using ZIP:', outer_zip, f'({os.path.getsize(outer_zip)/1e9:.2f} GB)')
    # Unzip outer wrapper (if not already)
    with zipfile.ZipFile(outer_zip) as zf:
        # Extract only tar.gz if present, otherwise extract all (first pass)
        tgz_names = [n for n in zf.namelist() if n.lower().endswith(('.tar.gz', '.tgz'))]
        if tgz_names:
            for n in tgz_names:
                target = os.path.join(IMNET_SRC, os.path.basename(n))
                if not os.path.exists(target):
                    print('Extracting inner tarball from ZIP:', n)
                    zf.extract(n, path=IMNET_SRC)
        else:
            print('No tarball found in ZIP; extracting all to locate ILSVRC...')
            zf.extractall(IMNET_SRC)

    # After unzip, extract any tar.gz
    tgzs = sorted(glob.glob(os.path.join(IMNET_SRC, '**', '*.tar.gz'), recursive=True))
    for tgz in tgzs:
        dest_flag = os.path.join(os.path.dirname(tgz), '.extracted_flag')
        if os.path.exists(dest_flag):
            continue
        print('Extracting tarball:', tgz)
        with tarfile.open(tgz, 'r:gz') as tf:
            tf.extractall(os.path.dirname(tgz))
        open(dest_flag, 'w').close()

    # Locate CLS-LOC root
    CLS_LOC_ROOT = ''
    for root, dirs, files in os.walk(IMNET_SRC):
        if root.endswith('/ILSVRC/Data/CLS-LOC/train') or root.endswith('/ILSVRC/Data/CLS-LOC/val'):
            CLS_LOC_ROOT = root.split('/ILSVRC/Data/CLS-LOC/')[0] + '/ILSVRC/Data/CLS-LOC'
            break
    print('CLS_LOC_ROOT:', CLS_LOC_ROOT or '<not found>')
    if CLS_LOC_ROOT:
        os.environ['CLS_LOC_ROOT'] = CLS_LOC_ROOT
        print('✓ Set env CLS_LOC_ROOT for downstream code.')
else:
    print('No ZIPs in IMNET_SRC; you can run the Kaggle cell below if needed.')


### Kaggle CLI reference
If you prefer running the raw command, use:

```bash
kaggle competitions download -c imagenet-object-localization-challenge -p "$IMNET_SRC"
unzip -q "$IMNET_SRC"/*.zip -d "$IMNET_SRC"
```

Note: ensure your `kaggle.json` is configured (`~/.kaggle/kaggle.json`) or upload it in the previous cell.


In [ ]:
# Kaggle download (optional)
# Installs Kaggle, configures credentials via kaggle.json upload, and downloads CLS-LOC files
!pip -q install kaggle
from google.colab import files
import os, glob, shutil

print('Upload kaggle.json from your Kaggle Account page (https://www.kaggle.com/<you>/account)')
uploaded = files.upload()  # choose kaggle.json
if 'kaggle.json' in uploaded:
    os.makedirs('/root/.kaggle', exist_ok=True)
    with open('/root/.kaggle/kaggle.json','wb') as f:
        f.write(uploaded['kaggle.json'])
    os.chmod('/root/.kaggle/kaggle.json', 0o600)
    os.environ['KAGGLE_CONFIG_DIR'] = '/root/.kaggle'
    print('kaggle.json installed at /root/.kaggle/kaggle.json')
else:
    print('kaggle.json not uploaded; if you already configured Kaggle, ignore this message.')

# Download competition data to IMNET_SRC (defined above)
os.makedirs(IMNET_SRC, exist_ok=True)
COMP = 'imagenet-object-localization-challenge'
print('Listing available files on Kaggle:')
!kaggle competitions files -c $COMP | head -n 50

# Use Kaggle Python API to find the main archive and download reliably
from kaggle import api

resp = api.competition_list_files(COMP)
# Get a plain list of file objects regardless of API model version
files = getattr(resp, "files", resp)

ALLOWED_EXTS = (".zip", ".tar.gz", ".tgz")
KNOWN_MAIN_NAMES = [
    "imagenet_object_localization_patched2019.tar.gz",
    "imagenet_object_localization_patched2017.tar.gz",
    "imagenet_object_localization_patched2016.zip",
]

def _filesize(f):
    # Kaggle models sometimes use different field names
    return getattr(f, "size", None) or getattr(f, "totalBytes", 0) or getattr(f, "bytes", 0)

def _filename(f):
    return f.name if hasattr(f, "name") else getattr(f, "fileName", None)

all_items = []
for f in list(files):
    name = _filename(f)
    if not name:
        continue
    sz = _filesize(f) or 0
    all_items.append((name, sz))

# 1) Prefer known filenames if present
chosen = None
names_lower = {n.lower(): n for (n, _) in all_items}
for known in KNOWN_MAIN_NAMES:
    if known.lower() in names_lower:
        chosen = names_lower[known.lower()]
        break

# 2) Otherwise, choose the largest allowed archive that is not an annotation/listing
if chosen is None:
    def good_candidate(item):
        n, sz = item
        ln = n.lower()
        if "/annotations/" in ln or ln.endswith(".xml") or ln.endswith(".txt"):
            return False
        ok_ext = ln.endswith(ALLOWED_EXTS[0]) or ln.endswith(ALLOWED_EXTS[1]) or ln.endswith(ALLOWED_EXTS[2])
        has_hint = ("localization" in ln) or ("cls-loc" in ln) or ("imagenet_object" in ln) or ("ilsvrc" in ln)
        return ok_ext and has_hint
    candidates = [it for it in all_items if good_candidate(it)]
    if candidates:
        candidates.sort(key=lambda it: it[1], reverse=True)  # by size desc
        chosen = candidates[0][0]

# 3) Last resort: pick the overall largest archive by extension
if chosen is None:
    archives = [it for it in all_items if it[0].lower().endswith(ALLOWED_EXTS[0]) or it[0].lower().endswith(ALLOWED_EXTS[1]) or it[0].lower().endswith(ALLOWED_EXTS[2])]
    if archives:
        archives.sort(key=lambda it: it[1], reverse=True)
        chosen = archives[0][0]

print("Chosen archive:", chosen)
assert chosen, "Could not detect a downloadable archive from Kaggle files list. Did you accept the competition rules?"

# Download that specific file
api.competition_download_file(COMP, chosen, path=IMNET_SRC, force=True, quiet=False)

# After API download, Kaggle wraps files as .zip when using CLI; API preserves name.zip
# Unzip any zips we have
zips = [p for p in glob.glob(os.path.join(IMNET_SRC, '*.zip')) if os.path.isfile(p)]
print('Found zip archives:', [os.path.basename(p) for p in zips])
for zp in zips:
    print('Unzipping', os.path.basename(zp))
    !unzip -n -q "$zp" -d "$IMNET_SRC"

# If we have a tar.gz/tgz inside, extract it
tgzs = [p for p in glob.glob(os.path.join(IMNET_SRC, '*.tar.gz')) + glob.glob(os.path.join(IMNET_SRC, '*.tgz')) if os.path.isfile(p)]
print('Found tarballs:', [os.path.basename(p) for p in tgzs])
for tgz in tgzs:
    print('Extracting', os.path.basename(tgz))
    !tar -xf "$tgz" -C "$IMNET_SRC"

# Final check for Kaggle layout
print('Checking for ILSVRC/Data/CLS-LOC...')
!find "$IMNET_SRC" -maxdepth 4 -type d -path "*/ILSVRC/Data/CLS-LOC/*" -print || true
print('Kaggle download complete at:', IMNET_SRC)


In [ ]:
# Verify ImageNet files (supports tarballs and Kaggle layout)
print("Checking for ImageNet files...")

# Option A: Original tarball format
tarball_files = ['ILSVRC2012_img_train.tar','ILSVRC2012_img_val.tar','ILSVRC2012_devkit_t12.tar']
tarball_missing = [f for f in tarball_files if not os.path.exists(os.path.join(IMNET_SRC,f))]

# Option B: Kaggle layout
kaggle_train_dir = os.path.join(IMNET_SRC, "ILSVRC", "Data", "CLS-LOC", "train")
kaggle_val_dir = os.path.join(IMNET_SRC, "ILSVRC", "Data", "CLS-LOC", "val")
USE_KAGGLE_LAYOUT = os.path.isdir(kaggle_train_dir) and os.path.isdir(kaggle_val_dir)

if not tarball_missing:
    print("✅ Found tarball format (original ImageNet files)")
    USE_KAGGLE_LAYOUT = False
elif USE_KAGGLE_LAYOUT:
    print("✅ Found Kaggle layout")
else:
    raise FileNotFoundError(
        f"Missing tarballs: {tarball_missing} and Kaggle layout not found under "
        f"{os.path.join(IMNET_SRC,'ILSVRC','Data','CLS-LOC')}"
    )


### Kaggle CLI reference
If you prefer running the raw command, use:

```bash
kaggle competitions download -c imagenet-object-localization-challenge -p "$IMNET_SRC"
unzip -q "$IMNET_SRC"/*.zip -d "$IMNET_SRC"
```

Note: ensure your `kaggle.json` is configured (`~/.kaggle/kaggle.json`) or upload it in the previous cell.


In [ ]:
# Prepare data: use Kaggle layout if available, else extract tarballs
import os, subprocess

kaggle_train_dir = os.path.join(IMNET_SRC, "ILSVRC", "Data", "CLS-LOC", "train")
kaggle_val_dir = os.path.join(IMNET_SRC, "ILSVRC", "Data", "CLS-LOC", "val")

os.makedirs(os.path.join(IMNET_ROOT, 'train'), exist_ok=True)
os.makedirs(os.path.join(IMNET_ROOT, 'val'), exist_ok=True)
os.makedirs(os.path.join(IMNET_ROOT, 'devkit'), exist_ok=True)

if 'USE_KAGGLE_LAYOUT' in globals() and USE_KAGGLE_LAYOUT:
    print("Using Kaggle layout → copying to target...")
    if not os.listdir(os.path.join(IMNET_ROOT,'train')):
        subprocess.run(['bash','-lc', f'cp -r "{kaggle_train_dir}"/* "{IMNET_ROOT}/train/"'], check=True)
    if not os.listdir(os.path.join(IMNET_ROOT,'val')):
        subprocess.run(['bash','-lc', f'cp -r "{kaggle_val_dir}"/* "{IMNET_ROOT}/val/"'], check=True)
else:
    print("Using tarballs → extracting...")
    train_tar = os.path.join(IMNET_SRC,'ILSVRC2012_img_train.tar')
    val_tar = os.path.join(IMNET_SRC,'ILSVRC2012_img_val.tar')
    devkit_tar = None
    for cand in ['ILSVRC2012_devkit_t12.tar','ILSVRC2012_devkit_t12.tar.gz']:
        p = os.path.join(IMNET_SRC, cand)
        if os.path.exists(p):
            devkit_tar = p; break
    if not os.listdir(os.path.join(IMNET_ROOT,'train')):
        subprocess.run(['bash','-lc', f'tar -xf "{train_tar}" -C "{IMNET_ROOT}/train"'], check=True)
        subprocess.run(['bash','-lc', 'cd "{}/train" && find . -name "*.tar" -print0 | xargs -0 -I{{}} bash -lc '\''d=$(basename "{}" .tar); mkdir -p "$d"; tar -xf "{}" -C "$d"; rm "{}"'\'''.format(IMNET_ROOT)], check=True)
    if not os.listdir(os.path.join(IMNET_ROOT,'val')):
        subprocess.run(['bash','-lc', f'tar -xf "{val_tar}" -C "{IMNET_ROOT}/val"'], check=True)
    if devkit_tar and not os.listdir(os.path.join(IMNET_ROOT,'devkit')):
        subprocess.run(['bash','-lc', f'tar -xf "{devkit_tar}" -C "{IMNET_ROOT}/devkit"'], check=True)
print("Done preparing data.")


In [ ]:
# Organize validation set into class folders
import os, shutil
import scipy.io as sio

devkit = '/content/imagenet/devkit/ILSVRC2012_devkit_t12'
meta = sio.loadmat(os.path.join(devkit,'data','meta.mat'))['synsets']
# Build mapping: ILSVRC2012_ID -> WNID
id2wnid = {}
for entry in meta:
    ILSVRC2012_ID = int(entry['ILSVRC2012_ID'][0][0]) if entry['ILSVRC2012_ID'].size else 0
    if ILSVRC2012_ID>0:
        wnid = str(entry['WNID'][0])
        id2wnid[ILSVRC2012_ID] = wnid

with open(os.path.join(devkit,'data','ILSVRC2012_validation_ground_truth.txt'),'r') as f:
    gt = [int(x.strip()) for x in f if x.strip()]

val_dir = '/content/imagenet/val'
imgs = sorted([x for x in os.listdir(val_dir) if x.lower().endswith(('.jpeg','.jpg'))])
assert len(imgs)==len(gt), f'mismatch: {len(imgs)} images vs {len(gt)} labels'

for img, cls_id in zip(imgs, gt):
    wnid = id2wnid[cls_id]
    dst = os.path.join(val_dir, wnid)
    os.makedirs(dst, exist_ok=True)
    shutil.move(os.path.join(val_dir,img), os.path.join(dst,img))


In [ ]:
# Run A/B/E for ViT-B/16 (~86M). Reduce batch if OOM.
IMAGENET_ROOT = '/content/imagenet'
!python experiments/imagenet_ab_param_budgets.py \
  --data_root $IMAGENET_ROOT \
  --targets 86000000 \
  --models A B E \
  --img_size 224 --patch 16 \
  --steps 90000 --eval_every 1000 --batch 128 \
  --lr_large 0.001 --warmup_frac 0.1 --weight_decay 0.1 \
  --use_randaug --randaug_n 2 --randaug_m 9 --random_erasing 0.25 \
  --mixup_alpha 0.8 --cutmix_alpha 1.0 --mix_prob 0.5 \
  --drop_path 0.4 --grad_clip 1.0 --ema --ema_decay 0.9999 \
  --ew_views 5 --ew_use_k3 --ew_share_qkv --ew_mlp_ratio 4.0


In [ ]:
# Optional: ViT-L/16 and ViT-H/14
!python experiments/imagenet_ab_param_budgets.py --data_root $IMAGENET_ROOT \
  --targets 307000000 --models A B E --img_size 224 --patch 16 --batch 128 \
  --lr_large 0.001 --warmup_frac 0.1 --weight_decay 0.1 --use_randaug --randaug_n 2 --randaug_m 9 \
  --random_erasing 0.25 --mixup_alpha 0.8 --cutmix_alpha 1.0 --mix_prob 0.5 \
  --drop_path 0.4 --grad_clip 1.0 --ema --ema_decay 0.9999 \
  --ew_views 5 --ew_use_k3 --ew_share_qkv --ew_mlp_ratio 4.0

!python experiments/imagenet_ab_param_budgets.py --data_root $IMAGENET_ROOT \
  --targets 632000000 --models A B E --img_size 224 --patch 14 --batch 128 \
  --lr_large 0.001 --warmup_frac 0.1 --weight_decay 0.1 --use_randaug --randaug_n 2 --randaug_m 9 \
  --random_erasing 0.25 --mixup_alpha 0.8 --cutmix_alpha 1.0 --mix_prob 0.5 \
  --drop_path 0.4 --grad_clip 1.0 --ema --ema_decay 0.9999 \
  --ew_views 5 --ew_use_k3 --ew_share_qkv --ew_mlp_ratio 4.0


In [ ]:
# Generate paper tables
!python experiments/ab5_paper_benchmark.py
!ls -la results/paper_benchmark
